In [ ]:
# Step 1: Imports
import tkinter as tk
from tkinter import ttk, messagebox
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import re

# -------------------- CONFIG --------------------
# Change path if needed (keep raw string if using Windows paths)
CSV_PATH = r"C:/Users/Siddharth Sumaria/OneDrive/Desktop/Data Science/CloudThat Projects/8 week project/My project Dataset/12 month ecommerce dataset.csv"   # e.g. r"C:\path\to\12 month ecommerce dataset.csv"
N_LAGS = 3
FUTURE_MONTHS = [13, 14]        # forecast two months ahead
# ------------------------------------------------

# Step 2: Load & Prepare Data (same pattern as yours, with a few guards)
df = pd.read_csv(CSV_PATH)

# Drop exact duplicates
df = df.drop_duplicates()

# Parse date
df['Order Date'] = pd.to_datetime(df['Order Date'], errors='coerce')
df = df.dropna(subset=['Order Date'])

# Normalize text columns
if 'Product' in df.columns:
    df['Product'] = df['Product'].astype(str).str.strip().str.title()
else:
    df['Product'] = 'Unknown'

# Coerce numerics for safety
for col in ['Quantity Ordered', 'Price Each']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

df = df.dropna(subset=['Quantity Ordered', 'Price Each'])

# Month
df['Month'] = df['Order Date'].dt.month

# Total sales value
df['Total Price'] = df['Quantity Ordered'] * df['Price Each']

# Add City if missing (simple split from Purchase Address)
def city_from_addr(addr):
    try:
        s = str(addr)
        parts = [p.strip() for p in s.split(',')]
        # Typical format: "Street, City, State ZIP"
        if len(parts) >= 2:
            return parts[-2]
        # Fallback: attempt regex "City, ST"
        m = re.search(r',\s*([^,]+),\s*[A-Z]{2}\b', s)
        return m.group(1).strip() if m else 'Unknown'
    except Exception:
        return 'Unknown'

if 'City' not in df.columns:
    if 'Purchase Address' in df.columns:
        df['City'] = df['Purchase Address'].apply(city_from_addr)
    else:
        df['City'] = 'Unknown'

# Categorize Product Type (same as yours)
def get_product_type(product):
    product = (product or "").lower()
    if 'phone' in product: return 'Phone'
    elif 'headphone' in product or 'ear' in product: return 'Audio'
    elif 'charger' in product or 'cable' in product: return 'Accessory'
    elif 'monitor' in product or 'screen' in product: return 'Display'
    elif 'macbook' in product or 'laptop' in product: return 'Laptop'
    elif 'keyboard' in product or 'mouse' in product: return 'Input'
    else: return 'Other'

df['Product Type'] = df['Product'].apply(get_product_type)

# Keep canonical months 1..12 only (historical window)
df = df[df['Month'].between(1, 12)]

# -------------------- Simple lists for dropdowns --------------------
cities = ['All Cities'] + sorted(df['City'].dropna().unique().tolist())
product_groups = ['All Types'] + sorted(df['Product Type'].dropna().unique().tolist())

# -------------------- TKINTER UI --------------------
root = tk.Tk()
root.title("📈 Sales Forecast (Simple)")
root.geometry("980x640")

top = tk.Frame(root)
top.pack(padx=10, pady=10, fill="x")

tk.Label(top, text="City:").pack(side="left")
city_var = tk.StringVar(value=cities[0])
city_dd = ttk.Combobox(top, textvariable=city_var, values=cities, state="readonly", width=30)
city_dd.pack(side="left", padx=8)

tk.Label(top, text="Product Type:").pack(side="left")
ptype_var = tk.StringVar(value=product_groups[0])
ptype_dd = ttk.Combobox(top, textvariable=ptype_var, values=product_groups, state="readonly", width=30)
ptype_dd.pack(side="left", padx=8)

metrics_lbl = tk.Label(root, text="MSE: — | R²: —", font=("Arial", 10))
metrics_lbl.pack(pady=(0, 4))

canvas = None

def run_pipeline_and_plot():
    global canvas

    # Filter by city
    d = df.copy()
    if city_var.get() != 'All Cities':
        d = d[d['City'] == city_var.get()]

    if d.empty:
        messagebox.showwarning("Warning", "No data for the selected city.")
        return

    # Aggregate & pivot (Month x Product Type -> Total Price)
    monthly_sales = d.groupby(['Month', 'Product Type'], as_index=False)['Total Price'].sum()
    pvt = monthly_sales.pivot(index='Month', columns='Product Type', values='Total Price').fillna(0.0)
    # Ensure complete months 1..12 present
    pvt = pvt.reindex(range(1, 13), fill_value=0.0)

    # Build lag features + current month
    cols = pvt.columns
    df_reset = pvt.reset_index()  # has 'Month' column

    X, y = [], []
    for i in range(N_LAGS, len(df_reset)):
        # take N_LAGS previous rows of all product-type columns, flatten
        lagged = df_reset.iloc[i - N_LAGS:i][cols].values.flatten()
        current_month = [int(df_reset.loc[i, 'Month'])]
        X.append(np.concatenate((lagged, current_month)))
        y.append(df_reset.iloc[i][cols].values)

    X, y = np.array(X), np.array(y)

    if len(X) < 3:
        messagebox.showwarning("Warning", "Not enough data after lagging. Try reducing N_LAGS or choose All Cities.")
        return

    # Train/test split (last 2 samples as test)
    X_train, y_train = X[:-2], y[:-2]
    X_test, y_test = X[-2:], y[-2:]

    # Train model (multi-output supported)
    rf = RandomForestRegressor(n_estimators=150, random_state=42)
    rf.fit(X_train, y_train)

    # Predict + metrics
     # Predict + metrics
    y_pred = rf.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)              # <- NEW: RMSE
    r2  = r2_score(y_test, y_pred)
    metrics_lbl.config(text=f"MSE: {mse:.2f} | RMSE: {rmse:.2f} | R²: {r2:.3f}")

    # Predict months 13 & 14 (rolling)
    # Start with the last N_LAGS rows of actuals
    input_data = df_reset.iloc[-N_LAGS:][cols].values.flatten()
    future_predictions = []
    for next_month in FUTURE_MONTHS:
        features = np.concatenate((input_data[-N_LAGS * len(cols):], [next_month]))
        pred = rf.predict([features])[0]              # shape: (n_cols,)
        future_predictions.append(pred)
        # Roll forward by appending the predicted row
        input_data = np.concatenate((input_data, pred))

    future_df = pd.DataFrame(future_predictions, columns=cols, index=FUTURE_MONTHS)
    future_df.index.name = "Month"
    future_df = future_df.round(2)

    # Combine for plotting
    full_sales = pd.concat([pvt, future_df], axis=0)

    # Clear previous canvas
    if canvas:
        canvas.get_tk_widget().destroy()
        canvas = None

    fig, ax = plt.subplots(figsize=(10.5, 5.6))

    choice = ptype_var.get()

    # Helper to draw historical solid + future dashed for a single series
    def plot_hist_future(series, label=None):
        hist_x = list(range(1, 13))
        fut_x  = FUTURE_MONTHS
        ax.plot(hist_x, series.loc[1:12].values, marker='o', label=label)
        ax.plot(fut_x, series.loc[FUTURE_MONTHS].values, marker='o', linestyle='--')

    if choice == 'All Types':
        for c in full_sales.columns:
            plot_hist_future(full_sales[c], label=c)
        ax.legend(title="Product Type", bbox_to_anchor=(1.02, 1), loc='upper left')
        city_txt = city_var.get()
        ax.set_title(f"Monthly Sales + Forecast (1–{max(full_sales.index)}) — {city_txt}")
    else:
        if choice not in full_sales.columns:
            messagebox.showwarning("Warning", f"No data for '{choice}' in this city.")
            return
        plot_hist_future(full_sales[choice], label=choice)
        ax.legend([choice], loc='upper left')
        ax.set_title(f"{choice}: Monthly Sales + Forecast — {city_var.get()}")

    ax.set_xlabel("Month")
    ax.set_ylabel("Total Sales")
    ax.grid(True)
    plt.tight_layout()

    canvas = FigureCanvasTkAgg(fig, master=root)
    canvas.draw()
    canvas.get_tk_widget().pack(pady=8)

btn = tk.Button(root, text="Show Chart", command=run_pipeline_and_plot, font=("Arial", 11))
btn.pack(pady=6)

root.mainloop()


C:\Users\Siddharth Sumaria\AppData\Local\Temp\ipykernel_14720\3809198744.py:26: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Order Date'] = pd.to_datetime(df['Order Date'], errors='coerce')
